# Guía Completa de smtplib y el módulo email

Esta guía cubre el uso de la librería estándar `smtplib` para enviar correos electrónicos con Python. Aprenderemos a enviar correos de texto plano, HTML, adjuntos y a manejar errores.

## ¿Qué es SMTP?

**SMTP** (Simple Mail Transfer Protocol) es el protocolo estándar para el envío de correos electrónicos.

**Puertos Comunes:**
*   **587**: Usado para conexiones cifradas con **TLS** (STARTTLS). Es el más recomendado actualmente.
*   **465**: Usado para conexiones cifradas con **SSL** implícito.
*   **25**: Puerto antiguo, a menudo bloqueado por proveedores de internet para evitar spam.

## Requisitos Previos (Importante)

Para enviar correos desde Python usando tu cuenta personal (Gmail, Outlook, etc.), necesitas configurar la seguridad:

1.  **Gmail**:
    *   Activa la **Verificación en 2 pasos**.
    *   Genera una **Contraseña de Aplicación** (App Password). *No uses tu contraseña normal.*
2.  **Outlook/Hotmail**:
    *   Similar a Gmail, busca "Contraseñas de aplicación" en la configuración de seguridad de Microsoft.

---
**Nota de Seguridad:** Nunca compartas tus credenciales en el código. En un entorno real, usa variables de entorno (`os.getenv`). Para este tutorial, usaremos variables, pero ten cuidado.

## 1. Importación de Librerías

Usaremos `smtplib` para la conexión y `email.message.EmailMessage` para construir el correo. `EmailMessage` es la forma moderna y preferida sobre la antigua `MIMEText`.

In [ ]:
import smtplib
from email.message import EmailMessage
import os
import mimetypes # Para detectar tipo de archivo adjunto

print("Librerías importadas.")

## 2. Configuración del Servidor

Aquí definimos las constantes.
**IMPORTANTE:** Si no tienes credenciales reales para probar, el código fallará al intentar autenticarse.

Para pruebas locales sin enviar correos reales, puedes usar una herramienta como `aiosmtpd` en otra terminal:
`python -m aiosmtpd -n -l localhost:1025`
Y configurar `SMTP_SERVER = 'localhost'` y `SMTP_PORT = 1025`.

In [ ]:
# Configuración para GMAIL
SMTP_SERVER = 'smtp.gmail.com'
SMTP_PORT = 587 # TLS

# Credenciales (Reemplaza con las tuyas o usa variables de entorno)
# EMAIL_USER = os.environ.get('EMAIL_USER')
# EMAIL_PASS = os.environ.get('EMAIL_PASS')

# Valores de ejemplo (NO FUNCIONARÁN sin credenciales reales)
EMAIL_USER = 'tu_correo@gmail.com'
EMAIL_PASS = 'tu_contraseña_de_aplicacion' 

DESTINATARIO = 'destinatario@ejemplo.com'

print(f"Configurado para conectar a {SMTP_SERVER}:{SMTP_PORT} como {EMAIL_USER}")

## 3. Envío de Email de Texto Plano

Este es el ejemplo más básico.
1.  Creamos `EmailMessage`.
2.  Seteamos `Subject`, `From`, `To`.
3.  Seteamos el contenido con `set_content()`.
4.  Conectamos, iniciamos TLS, nos logueamos y enviamos.

In [ ]:
def enviar_texto_plano():
    msg = EmailMessage()
    msg['Subject'] = 'Prueba de Python SMTP'
    msg['From'] = EMAIL_USER
    msg['To'] = DESTINATARIO
    msg.set_content('Hola,\n\nEste es un correo de prueba enviado desde Python.\n\nSaludos.')

    try:
        # Contexto SSL por defecto
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as smtp:
            smtp.starttls() # Encriptar la conexión
            smtp.login(EMAIL_USER, EMAIL_PASS)
            smtp.send_message(msg)
            print("Correo de texto plano enviado exitosamente.")
    except Exception as e:
        print(f"Error al enviar correo: {e}")

# Descomenta para probar (si tienes credenciales)
# enviar_texto_plano()

## 4. Envío de Email HTML

Para enviar correos con formato (negrita, colores, enlaces), usamos `add_alternative`.
Es buena práctica incluir una versión de texto plano primero (como fallback) y luego la versión HTML.

In [ ]:
def enviar_html():
    msg = EmailMessage()
    msg['Subject'] = 'Correo HTML desde Python'
    msg['From'] = EMAIL_USER
    msg['To'] = DESTINATARIO

    # 1. Contenido Texto Plano (Fallback)
    msg.set_content('Este cliente de correo no soporta HTML. Por favor habilítalo.')

    # 2. Contenido HTML
    html_content = """
    <!DOCTYPE html>
    <html>
        <body>
            <h1 style="color:Blue;">Hola desde Python!</h1>
            <p>Este es un correo con <b>formato HTML</b>.</p>
            <p>Visita <a href="https://www.python.org">Python.org</a></p>
        </body>
    </html>
    """
    msg.add_alternative(html_content, subtype='html')

    try:
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as smtp:
            smtp.starttls()
            smtp.login(EMAIL_USER, EMAIL_PASS)
            smtp.send_message(msg)
            print("Correo HTML enviado exitosamente.")
    except Exception as e:
        print(f"Error: {e}")

# enviar_html()

## 5. Adjuntar Archivos

Podemos adjuntar imágenes, PDFs, etc., usando `add_attachment`.
Es importante leer el archivo en modo binario (`'rb'`) y determinar su tipo MIME (ej. `image/jpeg`, `application/pdf`).

In [ ]:
def enviar_con_adjunto(ruta_archivo):
    msg = EmailMessage()
    msg['Subject'] = 'Correo con Adjunto'
    msg['From'] = EMAIL_USER
    msg['To'] = DESTINATARIO
    msg.set_content('Adjunto encontrarás el archivo solicitado.')

    # Adivinar tipo MIME
    ctype, encoding = mimetypes.guess_type(ruta_archivo)
    if ctype is None or encoding is not None:
        # No se pudo adivinar o está codificado, usar genérico
        ctype = 'application/octet-stream'
    
    maintype, subtype = ctype.split('/', 1)

    try:
        with open(ruta_archivo, 'rb') as f:
            file_data = f.read()
            file_name = os.path.basename(ruta_archivo)
            
            msg.add_attachment(file_data, 
                               maintype=maintype, 
                               subtype=subtype, 
                               filename=file_name)

        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as smtp:
            smtp.starttls()
            smtp.login(EMAIL_USER, EMAIL_PASS)
            smtp.send_message(msg)
            print(f"Correo con {file_name} enviado.")
            
    except FileNotFoundError:
        print("El archivo no existe.")
    except Exception as e:
        print(f"Error: {e}")

# Crear un archivo dummy para probar
with open('prueba.txt', 'w') as f:
    f.write('Contenido del archivo adjunto.')

# enviar_con_adjunto('prueba.txt')

## 6. Múltiples Destinatarios

Para enviar a varias personas, `msg['To']` puede recibir una cadena separada por comas o una lista (que luego unimos con `, `).
También podemos usar `msg['Cc']` y `msg['Bcc']` (Copia Oculta).

In [ ]:
def enviar_multiples():
    destinatarios = ['correo1@ejemplo.com', 'correo2@ejemplo.com']
    
    msg = EmailMessage()
    msg['Subject'] = 'Aviso Importante'
    msg['From'] = EMAIL_USER
    # Unir lista con comas
    msg['To'] = ', '.join(destinatarios) 
    msg.set_content('Hola a todos.')

    try:
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as smtp:
            smtp.starttls()
            smtp.login(EMAIL_USER, EMAIL_PASS)
            smtp.send_message(msg)
            print(f"Enviado a: {destinatarios}")
    except Exception as e:
        print(f"Error: {e}")

# enviar_multiples()

## 7. Manejo de Errores y Depuración

Es útil capturar excepciones específicas de SMTP y activar el modo debug para ver la comunicación con el servidor.

In [ ]:
def depuracion_smtp():
    try:
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as smtp:
            # Nivel 1 muestra la interacción con el servidor
            smtp.set_debuglevel(1) 
            
            smtp.starttls()
            smtp.login(EMAIL_USER, EMAIL_PASS)
            # ... enviar ...
            print("Conexión exitosa (modo debug).")
            
    except smtplib.SMTPAuthenticationError:
        print("Error de autenticación: Revisa tu usuario y contraseña (o App Password).")
    except smtplib.SMTPConnectError:
        print("Error de conexión: No se pudo conectar al servidor.")
    except Exception as e:
        print(f"Otro error: {e}")

# depuracion_smtp()